## Vertical regridding example
This notebook requires `intake`, `intake-esm` and `gcsfs` which are *not* dependencies of pyIC: you will need to install these.

This notebook is based on the `xgcm` example for regridding: https://xgcm.readthedocs.io/en/latest/transform.html#Realistic-Data-Example.

In [1]:
import intake
import xarray as xr
import numpy as np

import cf_xarray
from pyic.grid import GRID

In [2]:
#use intake to get some cmip6 data

col = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")

cat = col.search(
    source_id = 'CNRM-ESM2-1',
    member_id = 'r1i1p1f2',
    experiment_id = 'historical',
    variable_id= ['thetao','so','vo','areacello'],
    grid_label = 'gn'
)
ddict = cat.to_dataset_dict(zarr_kwargs={'consolidated':True, 'use_cftime':True}, aggregate=False)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version'


In [3]:
#construct datasets from cmip data

thetao = ddict['CMIP.CNRM-CERFACS.CNRM-ESM2-1.historical.r1i1p1f2.Omon.thetao.gn.gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1/historical/r1i1p1f2/Omon/thetao/gn/v20181206/.20181206']
so = ddict['CMIP.CNRM-CERFACS.CNRM-ESM2-1.historical.r1i1p1f2.Omon.so.gn.gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1/historical/r1i1p1f2/Omon/so/gn/v20181206/.20181206']
vo = ddict['CMIP.CNRM-CERFACS.CNRM-ESM2-1.historical.r1i1p1f2.Omon.vo.gn.gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1/historical/r1i1p1f2/Omon/vo/gn/v20181206/.20181206']
areacello = ddict['CMIP.CNRM-CERFACS.CNRM-ESM2-1.historical.r1i1p1f2.Ofx.areacello.gn.gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1/historical/r1i1p1f2/Ofx/areacello/gn/v20181206/.20181206'].areacello

vo = vo.rename({'y':'y_c', 'lon':'lon_v', 'lat':'lat_v', 'bounds_lon':'bounds_lon_v', 'bounds_lat':'bounds_lat_v'})

ds = xr.merge([thetao,so,vo], compat='override')
ds = ds.assign_coords(areacello=areacello.fillna(0))
ds

<xarray.Dataset> Size: 190GB
Dimensions:         (y: 294, x: 362, nvertex: 4, lev: 75, axis_nbounds: 2,
                     member_id: 1, dcpp_init_year: 1, time: 1980, y_c: 294)
Coordinates: (12/15)
    bounds_lat      (y, x, nvertex) float64 3MB dask.array<chunksize=(294, 362, 4), meta=np.ndarray>
    bounds_lon      (y, x, nvertex) float64 3MB dask.array<chunksize=(294, 362, 4), meta=np.ndarray>
    lat             (y, x) float64 851kB dask.array<chunksize=(294, 362), meta=np.ndarray>
  * lev             (lev) float64 600B 0.5058 1.556 ... 5.698e+03 5.902e+03
    lev_bounds      (lev, axis_nbounds) float64 1kB dask.array<chunksize=(75, 2), meta=np.ndarray>
    lon             (y, x) float64 851kB dask.array<chunksize=(294, 362), meta=np.ndarray>
    ...              ...
  * dcpp_init_year  (dcpp_init_year) float64 8B nan
    bounds_lat_v    (y_c, x, nvertex) float64 3MB dask.array<chunksize=(294, 362, 4), meta=np.ndarray>
    bounds_lon_v    (y_c, x, nvertex) float64 3MB dask.array<chunksize=(294, 362, 4), meta=np.ndarray>
    lat_v           (y_c, x) float64 851kB dask.array<chunksize=(294, 362), meta=np.ndarray>
    lon_v           (y_c, x) float64 851kB dask.array<chunksize=(294, 362), meta=np.ndarray>
    areacello       (member_id, dcpp_init_year, y, x) float32 426kB dask.array<chunksize=(1, 1, 294, 362), meta=np.ndarray>
Dimensions without coordinates: y, x, nvertex, axis_nbounds, y_c
Data variables:
    thetao          (member_id, dcpp_init_year, time, lev, y, x) float32 63GB dask.array<chunksize=(1, 1, 4, 75, 294, 362), meta=np.ndarray>
    so              (member_id, dcpp_init_year, time, lev, y, x) float32 63GB dask.array<chunksize=(1, 1, 5, 75, 294, 362), meta=np.ndarray>
    vo              (member_id, dcpp_init_year, time, lev, y_c, x) float32 63GB dask.array<chunksize=(1, 1, 3, 75, 294, 362), meta=np.ndarray>
Attributes: (12/66)
    CMIP6_CV_version:                 cv=6.2.3.0-7-g2019642
    Conventions:                      CF-1.7 CMIP-6.2
    EXPID:                            CNRM-ESM2-1_historical_r1i1p1f2
    activity_id:                      CMIP
    arpege_minor_version:             6.3.2
    branch_method:                    standard
    ...                               ...
    intake_esm_attrs:variable_id:     thetao
    intake_esm_attrs:grid_label:      gn
    intake_esm_attrs:zstore:          gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM...
    intake_esm_attrs:version:         20181206
    intake_esm_attrs:_data_format_:   zarr
    intake_esm_dataset_key:           CMIP.CNRM-CERFACS.CNRM-ESM2-1.historica...

In [4]:
#assign coords

level_outer_data = cf_xarray.bounds_to_vertices(ds.lev_bounds, 'axis_nbounds').load().data

ds = ds.assign_coords({'level_outer': level_outer_data})

In [5]:
#use vertical regrid in pyIC to produce regridded vertical

In [6]:
ds_rg = GRID(dataset=ds, convert_to_z_grid=True, z_kwargs={"variable":'all',
                                                   "target": np.arange(0,500.,50),
                                                   #"target_variable":"thetao",
                                                   'coord':'lev',
                                                    'method':'linear'
                                                  })

Vertical regridding is still under construction. Use at your own risk.


/home/users/jdconey/miniforge3/envs/pyIC_dev/lib/python3.11/site-packages/xgcm/grid.py:989: FutureWarning: From version 0.8.0 the Axis computation methods will be removed, in favour of using the Grid computation methods instead. i.e. use `Grid.transform` instead of `Axis.transform`
  warnings.warn(
/home/users/jdconey/miniforge3/envs/pyIC_dev/lib/python3.11/site-packages/xgcm/grid.py:989: FutureWarning: From version 0.8.0 the Axis computation methods will be removed, in favour of using the Grid computation methods instead. i.e. use `Grid.transform` instead of `Axis.transform`
  warnings.warn(
/home/users/jdconey/miniforge3/envs/pyIC_dev/lib/python3.11/site-packages/xgcm/grid.py:989: FutureWarning: From version 0.8.0 the Axis computation methods will be removed, in favour of using the Grid computation methods instead. i.e. use `Grid.transform` instead of `Axis.transform`
  warnings.warn(


In [7]:
#regridded
ds_rg.ds['so']

<xarray.DataArray 'so' (member_id: 1, dcpp_init_year: 1, time: 1980, y: 294,
                        x: 362, lev: 10)> Size: 8GB
dask.array<transpose, shape=(1, 1, 1980, 294, 362, 10), dtype=float32, chunksize=(1, 1, 5, 294, 362, 10), chunktype=numpy.ndarray>
Coordinates:
    lat             (y, x) float64 851kB dask.array<chunksize=(294, 362), meta=np.ndarray>
    lon             (y, x) float64 851kB dask.array<chunksize=(294, 362), meta=np.ndarray>
  * time            (time) object 16kB 1850-01-16 12:00:00 ... 2014-12-16 12:...
  * member_id       (member_id) object 8B 'r1i1p1f2'
  * dcpp_init_year  (dcpp_init_year) float64 8B nan
    areacello       (member_id, dcpp_init_year, y, x) float32 426kB dask.array<chunksize=(1, 1, 294, 362), meta=np.ndarray>
  * lev             (lev) float64 80B 0.0 50.0 100.0 150.0 ... 350.0 400.0 450.0
Dimensions without coordinates: y, x

In [8]:
#original
ds_rg.raw_ds['so']

<xarray.DataArray 'so' (member_id: 1, dcpp_init_year: 1, time: 1980, lev: 75,
                        y: 294, x: 362)> Size: 63GB
dask.array<broadcast_to, shape=(1, 1, 1980, 75, 294, 362), dtype=float32, chunksize=(1, 1, 5, 75, 294, 362), chunktype=numpy.ndarray>
Coordinates:
    lat             (y, x) float64 851kB dask.array<chunksize=(294, 362), meta=np.ndarray>
  * lev             (lev) float64 600B 0.5058 1.556 ... 5.698e+03 5.902e+03
    lon             (y, x) float64 851kB dask.array<chunksize=(294, 362), meta=np.ndarray>
  * time            (time) object 16kB 1850-01-16 12:00:00 ... 2014-12-16 12:...
  * member_id       (member_id) object 8B 'r1i1p1f2'
  * dcpp_init_year  (dcpp_init_year) float64 8B nan
    areacello       (member_id, dcpp_init_year, y, x) float32 426kB dask.array<chunksize=(1, 1, 294, 362), meta=np.ndarray>
Dimensions without coordinates: y, x
Attributes:
    cell_measures:       area: areacello volume: volcello
    cell_methods:        area: mean where sea time: mean
    description:         Sea Water Salinity
    history:             none
    interval_operation:  1800 s
    interval_write:      1 month
    long_name:           Sea Water Salinity
    online_operation:    average
    standard_name:       sea_water_salinity
    units:               0.001